In [61]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import healpy as hp
from astropy import units as u
from astropy.coordinates import SkyCoord

In [55]:
def show_pix(a, nside=2**11):
    m = np.zeros((hp.nside2npix(nside)), dtype=np.int32)
    i = 0
    for npix in a:
        vec = hp.pix2vec(nside=nside, ipix=npix, nest=True)
        ipix_disc = hp.query_disc(nside=nside, vec=vec, radius=np.radians(1), nest=True)
        m[ipix_disc] = 1 + i
        i += 1
    hp.mollview(m, title="Mollview image NEST", nest=True)

In [65]:
def make_pic(center_pix, size=64, nside=2**11):
    
    def get_neighbours(npix):
        theta, phi = hp.pix2ang(nside=nside, ipix=npix, nest=True)
        neighbours = hp.get_all_neighbours(nside=nside, theta=theta, phi=phi, nest=True)
        return neighbours
    
    
    picture_pixels_idx = [center_pix]
    picture_pixels_idx.extend(get_neighbours(center_pix)[:3])
    
    half = size // 2
    far_pix = None
    for i in range(half):
        next_idx = set(picture_pixels_idx)
        for pix in picture_pixels_idx:
            next_idx = next_idx.union(set(get_neighbours(pix)))
            
        if i == half - 1:
            far_pix = next_idx.difference(picture_pixels_idx)
        else:
            picture_pixels_idx = next_idx
    
    far_pix = list(far_pix)
    picture_pixels_idx = list(picture_pixels_idx)
    
    
    def dist_pixels(apix, bpix):
        theta, phi = hp.pix2ang(nest=True, nside=nside, ipix=apix)
        a = SkyCoord(frame='galactic', l=theta*u.radian, b=phi*u.radian)
        theta, phi = hp.pix2ang(nest=True, nside=nside, ipix=bpix)
        b = SkyCoord(frame='galactic', l=theta*u.radian, b=phi*u.radian)
        return a.separation(b).degree
    
    dist = max(dist_pixels(center_pix, far_pix))
    
    return picture_pixels_idx, dist

In [66]:
idx, dist = make_pic(center_pix=6, nside=2**11)
print(dist)

1.4501953124999987
